![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Taller: Construcción e implementación de modelos Bagging, Random Forest y XGBoost

En este taller podrán poner en práctica sus conocimientos sobre la construcción e implementación de modelos de Bagging, Random Forest y XGBoost. El taller está constituido por 8 puntos, en los cuales deberan seguir las intrucciones de cada numeral para su desarrollo.

## Datos predicción precio de automóviles

En este taller se usará el conjunto de datos de Car Listings de Kaggle donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [73]:
# Importación de librerías
%matplotlib inline
import random
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from xgboost import XGBRegressor

In [56]:
# Lectura de la información de archivo .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')

# Preprocesamiento de datos para el taller
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data = data.drop(['Model'], axis=1)

# Visualización dataset
data.head()

,Price,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE
7,21995,2014,6480,0,0,0,1,0,0,0
11,13995,2014,39972,0,0,0,0,1,0,0
167,17941,2016,18989,0,0,0,0,0,1,0
225,12493,2014,51330,0,0,0,1,0,0,0
270,7994,2007,116065,0,1,0,0,0,0,0


In [3]:
# Separación de variables predictoras (X) y variable de interés (y)
y = data['Price']
X = data.drop(['Price'], axis=1)

In [4]:
# Separación de datos en set de entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Punto 1 - Árbol de decisión manual

En la celda 1 creen un árbol de decisión **manualmente**  que considere los set de entrenamiento y test definidos anteriormente y presenten el RMSE y MAE del modelo en el set de test.

In [104]:
def evaluate_model(y, y_pred):
    rmse = np.sqrt(np.mean((y_pred - y) ** 2))
    mae = np.mean(np.abs(y_pred - y))
    mape = np.mean(np.abs((y - y_pred) / y)) * 100
    
    return f"RMSE: {rmse:.2f} | MAE: {mae:.2f} | MAPE: {mape:.2f}%"

In [115]:
class DecisionTreeRegressorManual:
    def __init__(self, min_gain=0.001, max_depth=None, num_pct=10):
        self.min_gain = min_gain
        self.max_depth = max_depth
        self.num_pct = num_pct
        self.final_tree_ = None


    # Definición de la función que calcula el gini index
    def _gini(self, y):
        return 0 if y.shape[0] == 0 else 1 - (y.mean()**2 + (1 - y.mean())**2)

    # Definición de la función gini_imputiry para calular la ganancia de una variable predictora j dado el punto de corte k
    def _gini_impurity(self, X_col, y, split):
        filter_l = X_col < split
        y_l = y.loc[filter_l]
        y_r = y.loc[~filter_l]
        
        n_l = y_l.shape[0]
        n_r = y_r.shape[0]
        
        gini_y = self._gini(y)
        gini_l = self._gini(y_l)
        gini_r = self._gini(y_r)
        
        gini_impurity_ = gini_y - (n_l / (n_l + n_r) * gini_l + n_r / (n_l + n_r) * gini_r)
        return gini_impurity_

    # Definición de la función best_split para calcular cuál es la mejor variable y punto de cortepara hacer la bifurcación del árbol
    def _best_split(self, X, y):
        features = range(X.shape[1])
        best_split = [0, 0, 0]  # j, split, gain
        
        # Para todas las varibles 
        for j in features:
            splits = np.percentile(X.iloc[:, j], np.arange(0, 100, 100.0 / (self.num_pct+1)).tolist())
            splits = np.unique(splits)[1:]
            
            # Para cada partición
            for split in splits:
                gain = self._gini_impurity(X.iloc[:, j], y, split)
                            
                if gain > best_split[2]:
                    best_split = [j, split, gain]
        
        return best_split

    # Definición de la función tree_grow para hacer un crecimiento recursivo del árbol
    def _tree_grow(self, X, y, level=0):
        # Si solo es una observación
        if X.shape[0] == 1:
            tree = dict(y_pred=y.iloc[:1].values[0], y_prob=0.5, level=self.selevel, split=-1, n_samples=1, gain=0)
            return tree
        
        # Calcular la mejor división
        j, split, gain = self._best_split(X, y)
        
        # Guardar el árbol y estimar la predicción
        y_pred = int(y.mean() >= 0.5) 
        y_prob = (y.sum() + 1.0) / (y.shape[0] + 2.0)  # Corrección Laplace 
        
        tree = dict(y_pred=y_pred, y_prob=y_prob, level=level, split=-1, n_samples=X.shape[0], gain=gain)
        
        # Revisar el criterio de parada 
        if gain < self.min_gain:
            return tree
        if self.max_depth is not None:
            if level >= self.max_depth:
                return tree
        
        # Continuar creando la partición
        filter_l = X.iloc[:, j] < split
        X_l, y_l = X.loc[filter_l], y.loc[filter_l]
        X_r, y_r = X.loc[~filter_l], y.loc[~filter_l]
        tree['split'] = [j, split]
        
        # Siguiente iteración para cada partición
        tree['sl'] = self._tree_grow(X_l, y_l, level + 1)
        tree['sr'] = self._tree_grow(X_r, y_r, level + 1)
        
        return tree
    
    def fit(self, X, y):
        self.final_tree_ = self._tree_grow(X, y)

    # Definición de la función tree_predict para hacer predicciones según las variables 'X' y el árbol 'tree'
    def _tree_predict(self, X, tree, proba=False):
        predicted = np.ones(X.shape[0])

        # Revisar si es el nodo final
        if tree['split'] == -1:
            if not proba:
                predicted = predicted * tree['y_pred']
            else:
                predicted = predicted * tree['y_prob']
        else:
            j, split = tree['split']
            filter_l = (X.iloc[:, j] < split)
            X_l = X.loc[filter_l]
            X_r = X.loc[~filter_l]

            if X_l.shape[0] == 0:  # Si el nodo izquierdo está vacio solo continua con el derecho 
                predicted[~filter_l] = self._tree_predict(X_r, tree['sr'], proba)
            elif X_r.shape[0] == 0:  #  Si el nodo derecho está vacio solo continua con el izquierdo
                predicted[filter_l] = self._tree_predict(X_l, tree['sl'], proba)
            else:
                predicted[filter_l] = self._tree_predict(X_l, tree['sl'], proba)
                predicted[~filter_l] = self._tree_predict(X_r, tree['sr'], proba)

        return predicted
    
    def predict(self, X, proba=False):
        return self._tree_predict(X, self.final_tree_, proba)


In [116]:
# Celda 1
model_dtr = DecisionTreeRegressorManual(min_gain=0.001, max_depth=6, num_pct=3)  
model_dtr.fit(X_train, y_train)

y_pred = model_dtr.predict(X_test, proba=True)
evaluate_model(y_test, y_pred)

'RMSE: 1664.79 | MAE: 1218.03 | MAPE: 8.68%'

### Punto 2 - Bagging manual

En la celda 2 creen un modelo bagging **manualmente** con 10 árboles de regresión y comenten sobre el desempeño del modelo.

In [80]:
# Celda 2
class BaggingRegressorManual:
    def __init__(self, estimator, n_estimators=10, random_state=None):
        self.estimator = estimator
        self.n_estimators = n_estimators
        self.random_state = random_state
        self.estimators_ = []

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X.iloc[indices], y.iloc[indices]

    def fit(self, X, y):
        random.seed(self.random_state)

        for _ in range(self.n_estimators):
            X_sample, y_sample = self._bootstrap_sample(X, y)
            self.estimator.fit(X_sample, y_sample)
            self.estimators_.append(self.estimator)

    def predict(self, X):
        X = X.values
        predictions = np.zeros((X.shape[0], self.n_estimators))

        for i, estimator in enumerate(self.estimators_):
            predictions[:, i] = estimator.predict(X)

        return np.mean(predictions, axis=1)

In [82]:
features_count = X.shape[1]
model_dtr =  DecisionTreeRegressor(max_features=int(np.log(features_count)))
bagreg = BaggingRegressorManual(model_dtr, n_estimators=10, random_state=1)
bagreg.fit(X_train, y_train)
y_pred = bagreg.predict(X_test)

evaluate_model(y_test, y_pred)

'RMSE: 2166.07 | MAE: 1587.10 | MAPE: 11.50%'

### Punto 3 - Bagging con librería

En la celda 3, con la librería sklearn, entrenen un modelo bagging con 10 árboles de regresión y el parámetro `max_features` igual a `log(n_features)` y comenten sobre el desempeño del modelo.

In [8]:
# Celda 3
features_count = X.shape[1]
model_dtr = DecisionTreeRegressor(max_features=int(np.log(features_count)))
bagreg = BaggingRegressor(model_dtr, n_estimators=10, bootstrap=True, oob_score=True, random_state=1)
bagreg.fit(X_train, y_train)
y_pred = bagreg.predict(X_test)

evaluate_model(y_test, y_pred)

'RMSE: 1824.29 | MAE: 1361.48 | MAPE: 9.87%'

### Punto 4 - Random forest con librería

En la celda 4, usando la librería sklearn entrenen un modelo de Randon Forest para regresión  y comenten sobre el desempeño del modelo.

In [9]:
# Celda 4
clf = RandomForestRegressor(n_estimators=10, max_features=int(np.log(features_count)), random_state=1, n_jobs=-1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
evaluate_model(y_test, y_pred)

'RMSE: 1819.56 | MAE: 1358.03 | MAPE: 9.87%'

In [10]:
pd.DataFrame({'feature':X.columns, 'importance':clf.feature_importances_}).sort_values('importance', ascending=False)

,feature,importance
1,Mileage,0.473487
0,Year,0.413996
3,M_Camry4dr,0.057410
7,M_CamrySE,0.015667
8,M_CamryXLE,0.014552
2,M_Camry,0.011398
6,M_CamryLE,0.006733
4,M_CamryBase,0.004620
5,M_CamryL,0.002137


### Punto 5 - Calibración de parámetros Random forest

En la celda 5, calibren los parámetros max_depth, max_features y n_estimators del modelo de Randon Forest para regresión, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [46]:
# Funciones necesarias
def plot_mape(ranges, mapes, xtitle = 'n_estimators'):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=(np.array(ranges)), y=mapes, mode='lines', name='MAPE'))
    fig.update_layout(xaxis_title=xtitle, yaxis_title='MAPE')
    pio.show(fig)

In [54]:
# Celda 5
# Evaluar el mejor valor para el parámetro n_estimators
estimator_range = range(10, 310, 10) # Entre 10 y 300 estimadores

# Para almacenar el porcentaje medio de error absolute MAPE de cada estimador
mapes = []

for estimator in estimator_range:
    clf = RandomForestRegressor(n_estimators=estimator, random_state=1, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mapes.append(mape)

display(pd.DataFrame({'n_estimator': estimator_range, 'MAPE': mapes}).sort_values(by='MAPE').head(10))
plot_mape(estimator_range, mapes)

,n_estimator,MAPE
28,290,9.503241
6,70,9.503359
23,240,9.503417
29,300,9.504331
25,260,9.504427
27,280,9.504476
22,230,9.504550
11,120,9.504662
15,160,9.504696
16,170,9.505250


In [55]:
# Evaluar el mejor valor para el parámetro max_features
feature_range = range(1, len(X.columns)+1)

# Para almacenar el porcentaje medio de error absolute MAPE de cada estimador
mapes = []

for feature in feature_range:
    clf = RandomForestRegressor(n_estimators=70, max_features=feature, random_state=1, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mapes.append(mape)

display(pd.DataFrame({'max_features': feature_range, 'MAPE': mapes}).sort_values(by='MAPE'))
plot_mape(feature_range, mapes, xtitle='max_features')

,max_features,MAPE
8,9,9.503359
7,8,9.556317
6,7,9.577025
5,6,9.626896
4,5,9.654582
3,4,9.684976
1,2,9.694288
2,3,9.696710
0,1,9.718459


> Los parámetros seleccionados son **n_stimator** de **70** y **max_features** de **9**.

In [49]:
clf = RandomForestRegressor(n_estimators=70, max_features=9, random_state=1, n_jobs=-1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
evaluate_model(y_test, y_pred)

'RMSE: 1757.24 | MAE: 1307.34 | MAPE: 9.50%'

In [14]:
# Prueba 2 - Optimizando con gridSearchCV
rf = RandomForestRegressor(random_state=1, n_jobs=-1)

# Especificar la matríz de parámetros
param_grid = {
    'n_estimators': (np.arange(10, 200, 10)),
    'max_features': [int(np.log(features_count))],
    'max_depth': (np.arange(2, 10, 1))
}

grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros encontrados
best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)


In [15]:
# # Crear un nuevo modelo con los mejores parámetros
# rf_best = RandomForestRegressor(**best_params, random_state=1, n_jobs=-1)
# rf_best.fit(X_train, y_train)
# y_pred = rf_best.predict(X_test)

# evaluate_model(y_test, y_pred)

### Punto 6 - XGBoost con librería

En la celda 6 implementen un modelo XGBoost de regresión con la librería sklearn y comenten sobre el desempeño del modelo.

In [16]:
# Celda 6
clf = XGBRegressor()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate_model(y_test, y_pred)

'RMSE: 1621.42 | MAE: 1186.63 | MAPE: 8.65%'

### Punto 7 - Calibración de parámetros XGBoost

En la celda 7 calibren los parámetros learning rate, gamma y colsample_bytree del modelo XGBoost para regresión, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [22]:
# Celda 7
# Valores a evaluar para cada parámetro ❓Probar con mas parametros
param_grid = {
    'learning_rate': [0.01, 0.1, 0.5],
    'gamma': [0, 0.1, 1],
    'colsample_bytree': [0.5, 0.7, 1]
}

# Evaluar cada modelo usando validación cruzada
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Mejores hiperparámetros encontrados:", grid_search.best_params_)


Mejores hiperparámetros encontrados: {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.1}


In [23]:
clf = XGBRegressor(colsample_bytree=0.5, gamma=0, learning_rate=0.1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate_model(y_test, y_pred)

'RMSE: 1549.65 | MAE: 1138.27 | MAPE: 8.28%'

### Punto 8 - Comparación y análisis de resultados
En la celda 8 comparen los resultados obtenidos de los diferentes modelos (random forest y XGBoost) y comenten las ventajas del mejor modelo y las desventajas del modelo con el menor desempeño.

In [19]:
# Celda 8
